# Data viz for paper submission

In [ ]:
# quick prep for proper ipynb viz in VS Code
!uv pip install ipykernel jupyter matplotlib seaborn

In [ ]:
# All imports
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Sec 1: Introduction

### Idk yet, probably want a catchy graph opposite abstract!

## Sec 2: Related Works

### Figure 2: Idk, related works probs won't need one?

## Sec 3: Existing Approaches for Mitigating Safety Degradation

In [ ]:
# helper function that retrieves scores on safety benchmarks produced by allenai/safety-eval
def retrieve_results(
    results_json_path: str,
    benchmark_configs: list[tuple[str, str]],
    compute_mean: bool = True,
    ):
    """
    Given a string filepath to a JSON file containing results created by safety-eval, and tuples of benchmarks and requested metrics, returns results as list and optionally the mean at the end.

        Args:
            results_json_path (`str`):
                Filepath of safety-eval generated JSON results file.
            benchmark_configs (`list[tuple[str, str]]`):
                List of tuples, each containing a benchmark and associated metric name. 
            compute_mean (`bool`, defaults to `False`):
                Whether or not to append average score at the end.

        Returns:
            Safety benchmark results as a list
    """
    with open(results_json_path) as fp:
        results_dict = json.load(fp)

    scores = []
    for benchmark, score_name in benchmark_configs:
        scores.append(results_dict[benchmark][score_name])
    
    if compute_mean:
        mean = sum(scores) / len(scores)
        scores.append(mean)
        # print(f"Average score for {results_json_path} is {mean}")
    return scores



### Sec 3.1: Lower Learning Rates During Supervised Fine-tuning

In [ ]:
# data for MATH-500, IFeval, IFBench from lighteval
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.container import BarContainer

RESULTS = {
    #  Model name                           Version      MATH-500           IFEval             IFBench
    "Base":                                 ("Base",     (0.460, 0.0223),  (0.7024, 0.0197),  (0.2433, 0.0248)),
    "Best low LR SFT":                      ("v00.24",   (0.598, 0.0219),  (0.4843, 0.0215),  (0.1267, 0.0192)),
    "Best Replay":                          ("v00.15",   (0.520, 0.0224),  (0.5619, 0.0214),  (0.1533, 0.0208)),
    "Best DFT":                             ("v00.17",   (0.424, 0.0221),  (0.3771, 0.0209),  (0.1067, 0.0179)),
    "Best EMA eta=0.75":                    ("v00.30",   (0.580, 0.0221),  (0.4695, 0.0215),  (0.1267, 0.0192)),
    "Best CISPO KL_beta=0.001":             ("v00.12",   (0.582, 0.0221),  (0.6784, 0.0201),  (0.2200, 0.0240)),
    "Best CISPO KL_beta=0":                 ("v00.14",   (0.594, 0.0220),  (0.6359, 0.0207),  (0.1900, 0.0227)),
}

BENCHMARKS = ["MATH-500", "IFEval", "IFBench"]

**Figure 3: Low LR results - Math**

**Figure 3: Low LR results - Utility**

**Figure 3: Low LR results - Safety**

In [ ]:
root_file_path = "../safety-eval/generation_results/"
model = "Neelectric/Llama-3.1-8B-Instruct_SFT_Math-220kv00."

safety_benchmark_configs = [
    ('wildguardtest', 'inverted_micro_harm_lower'),
    ('harmbench', 'inverted_micro_asr_lower'),
    ('toxigen', 'safe_overall'),
    ('do_anything_now', 'inverted_macro_asr'),
    ('wildjailbreak:harmful', 'inverted_macro_asr'),
    ('trustllm_jailbreaktrigger', 'inverted_macro_asr')
]
benchmarks = [benchmark[0] for benchmark in safety_benchmark_configs] + ["Average"]
print(benchmarks)

### do it for base
base_path = root_file_path + "meta-llama/Llama-3.1-8B-Instruct" + "/base/metrics_base_main.json"
base_values = retrieve_results(results_json_path=base_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
low_lr_values = []
for i in range(7,13):
    if i == 11: # the v00.11 run had a bug
        continue
    version_id_unpadded = str(i)
    version_id = "0" + version_id_unpadded if len(version_id_unpadded) == 1 else version_id_unpadded # turning "7" into "07"
    low_lr_path = root_file_path + model + version_id + "/SFT/metrics_SFT_main.json"
    low_lr_values += retrieve_results(results_json_path=low_lr_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
n = 6
print(len(benchmarks * n))
print(len(base_values + low_lr_values))
print(len(['base'] * len(benchmarks) + ['LR: 5e-5 (v00.07)'] * len(benchmarks) + ['LR: 1e-5 (v00.08)'] * len(benchmarks) + ['LR: 5e-6 (v00.09)'] * len(benchmarks)  + ['LR: 1e-6 (v00.10)'] * len(benchmarks) + ['LR: 5e-7 (v00.12)'] * len(benchmarks)))

df = pd.DataFrame({
    'Benchmark': benchmarks * n,
    'Value': base_values + low_lr_values,
    'Model': ['base'] * len(benchmarks) + ['LR: 5e-5 (v00.07)'] * len(benchmarks) + ['LR: 1e-5 (v00.08)'] * len(benchmarks) + ['LR: 5e-6 (v00.09)'] * len(benchmarks)  + ['LR: 1e-6 (v00.10)'] * len(benchmarks) + ['LR: 5e-7 (v00.12)'] * len(benchmarks)
})

fig, ax = plt.subplots(figsize=(14, 6))
palette = sns.color_palette("viridis", n)
bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette)

# again we add value labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f', padding=3, fontsize=6)

ax.set_xlabel('')
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Safety Benchmark Results\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim(0, 1 * 1.07) # trying to give room for labels
plt.tight_layout()
plt.show()

### Sec 3.2: Performing Replay On Refusals

**Figure x: Low percentage replay results**

In [ ]:
# data for MATH-500, IFeval, IFBench from lighteval
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.container import BarContainer

RESULTS = {
    #  Model name                                   Version      MATH-500           IFEval             IFBench
    "Base":                                         ("Base",     (0.460, 0.0223),  (0.7024, 0.0197),  (0.2433, 0.0248)),
    "Best low LR SFT":                              ("v00.24",   (0.598, 0.0219),  (0.4843, 0.0215),  (0.1267, 0.0192)),
    "1% Replay":                                    ("v00.32",   (0.566, 0.0222),  (0.3604, 0.0207),  (0.1467, 0.0205)),
    "2% Replay":                                    ("v00.33",   (0.566, 0.0222),  (0.4233, 0.0213),  (0.1600, 0.0212)),
    "3% Replay":                                    ("v00.34",   (0.582, 0.0221),  (0.4344, 0.0213),  (0.1767, 0.0221)),
    "4% Replay":                                    ("v00.35",   (0.560, 0.0222),  (0.4455, 0.0214),  (0.1867, 0.0225)),
    "5% Replay":                                    ("v00.13",   (0.522, 0.0224),  (0.0001, 0.0001),  (0.0001, 0.0001)),
}

BENCHMARKS = ["MATH-500", "IFEval", "IFBench"]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.container import BarContainer

rows = []
for name, (version, *scores) in RESULTS.items():
    label = f"{name} ({version})" if name != "Base" else "Base"
    for bench, (val, stderr) in zip(BENCHMARKS, scores):
        rows.append({"Model": label, "Benchmark": bench, "Value": val, "Stderr": stderr})

df = pd.DataFrame(rows)

# we select only instruction following benchmarks
df = df[df['Benchmark'].isin(['IFEval','IFBench'])]
fig, ax = plt.subplots(figsize=(8, 6))
n_models = len(df['Model'].unique())
palette = sns.color_palette("viridis", n_models)

bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette, errorbar=None)

benchmarks = df['Benchmark'].unique()
models = df['Model'].unique()

for i, container in enumerate(ax.containers):
    if isinstance(container, BarContainer):
        for j, bar in enumerate(container):
            model = models[i]
            benchmark = benchmarks[j]
            stderr = df[(df['Model'] == model) & (df['Benchmark'] == benchmark)]['Stderr'].values[0]
            x = bar.get_x() + bar.get_width() / 2
            y = bar.get_height()
            if y > 0:  # Only add error bars for non-zero values
                ax.errorbar(x, y, yerr=stderr, fmt='none',
                           color='black', capsize=3, capthick=1.5,
                           elinewidth=1.5, alpha=0.7, zorder=10)

# adding value labels on top of bars again
for container in ax.containers:
    if isinstance(container, BarContainer):
        labels = [f'{v.get_height():.3f}' if v.get_height() > 0 else '' for v in container]
        ax.bar_label(container, labels=labels, padding=15, fontsize=8, rotation=0)

ax.set_xlabel('Benchmark (Prompt-level strict accuracy)', fontsize=12)
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Instruction-Following Results after Replay\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
# ax.legend(title='Model', frameon=True, fancybox=True, shadow=True, bbox_to_anchor=(1.02, 1), loc='upper left')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Set y-axis limit to give room for labels and error bars
ax.set_ylim(0, max(df['Value'] + df['Stderr']) * 1.25)

plt.tight_layout()
plt.show()

**Figure 4: Replay results**

### Sec 3.3: Dynamic Fine-tuning

**Figure 5: Dynamic Fine-tuning results**

### Sec 3.4: Exponential Moving Average

**Figure 6: Exponential Moving Average results - safety**

In [ ]:
root_file_path = "../safety-eval/generation_results/"
model = "Neelectric/Llama-3.1-8B-Instruct_SFT_Math-220kv00."

safety_benchmark_configs = [
    ('wildguardtest', 'inverted_micro_harm_lower'),
    ('harmbench', 'inverted_micro_asr_lower'),
    ('toxigen', 'safe_overall'),
    ('do_anything_now', 'inverted_macro_asr'),
    ('wildjailbreak:harmful', 'inverted_macro_asr'),
    ('trustllm_jailbreaktrigger', 'inverted_macro_asr')
]
benchmarks = [benchmark[0] for benchmark in safety_benchmark_configs] + ["Average"]
print(benchmarks)

### do it for base
base_path = root_file_path + "meta-llama/Llama-3.1-8B-Instruct" + "/base/metrics_base_main.json"
base_values = retrieve_results(results_json_path=base_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
low_lr_values = []
for i in range(28,31):
    version_id_unpadded = str(i)
    version_id = "0" + version_id_unpadded if len(version_id_unpadded) == 1 else version_id_unpadded # turning "7" into "07"
    low_lr_path = root_file_path + model + version_id + "/SFT/metrics_SFT_main.json"
    low_lr_values += retrieve_results(results_json_path=low_lr_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
n = 4
print(len(benchmarks * n))
print(len(base_values + low_lr_values))
print(len(['base'] * len(benchmarks) + ['EMA eta: 0.25 (v00.28)'] * len(benchmarks) + ['EMA eta: 0.4 (v00.29)'] * len(benchmarks) + ['EMA eta: 0.75 (v00.30)'] * len(benchmarks) ))

df = pd.DataFrame({
    'Benchmark': benchmarks * n,
    'Value': base_values + low_lr_values,
    'Model': ['base'] * len(benchmarks) + ['EMA eta: 0.25 (v00.28)'] * len(benchmarks) + ['EMA eta: 0.4 (v00.29)'] * len(benchmarks) + ['EMA eta: 0.75 (v00.30)'] * len(benchmarks)
})

fig, ax = plt.subplots(figsize=(14, 6))
palette = sns.color_palette("viridis", n)
bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette)

# again we add value labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f', padding=3, fontsize=6)

ax.set_xlabel('')
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Safety Benchmark Results\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim(0, 1 * 1.07) # trying to give room for labels
plt.tight_layout()
plt.show()

### Sec 3.5: Reinforcement Learning with Verifiable Rewards

**Figure 7: RLVR results**

### Sec 3.6: Comparison of Best Checkpoints

In [ ]:
print("CURRENTLY HARDCODING VALS, SHOULD CHANGE THIS BEFORE SUBMISSION!")
print("IDEALLY A BENCHMARKING SCRIPT THAT CREATES ALL RESULTS FOR ALL FIGURES INDIVIDUALLY WHICH YOU CAN RUN BEFORE RUNNING ALL THESE!")

In [ ]:
# data for MATH-500, IFeval, IFBench from lighteval
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.container import BarContainer

RESULTS = {
    #  Model name                           Version      MATH-500           IFEval             IFBench
    "Base":                                 ("Base",     (0.460, 0.0223),  (0.7024, 0.0197),  (0.2433, 0.0248)),
    "Best low LR SFT":                      ("v00.24",   (0.598, 0.0219),  (0.4843, 0.0215),  (0.1267, 0.0192)),
    "Best Replay":                          ("v00.15",   (0.520, 0.0224),  (0.5619, 0.0214),  (0.1533, 0.0208)),
    "Best DFT":                             ("v00.17",   (0.424, 0.0221),  (0.3771, 0.0209),  (0.1067, 0.0179)),
    "Best EMA eta=0.75":                    ("v00.30",   (0.580, 0.0221),  (0.4695, 0.0215),  (0.1267, 0.0192)),
    "Best CISPO KL_beta=0.001":             ("v00.12",   (0.582, 0.0221),  (0.6784, 0.0201),  (0.2200, 0.0240)),
    "Best CISPO KL_beta=0":                 ("v00.14",   (0.594, 0.0220),  (0.6359, 0.0207),  (0.1900, 0.0227)),
}

BENCHMARKS = ["MATH-500", "IFEval", "IFBench"]

### Figure 8: Math Best checkpoint comparison graphs

In [ ]:
rows = []
for name, (version, *scores) in RESULTS.items():
    label = f"{name} ({version})" if name != "Base" else "Base"
    for bench, (val, stderr) in zip(BENCHMARKS, scores):
        rows.append({"Model": label, "Benchmark": bench, "Value": val, "Stderr": stderr})

df = pd.DataFrame(rows)
df = df[df['Benchmark'].isin(['MATH-500'])]

df['Benchmark'] = 'MATH-500 (pass@k, k=1 & n=1)'

fig, ax = plt.subplots(figsize=(6, 7))
n = len(df)
palette = sns.color_palette("viridis", n)

bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette, errorbar=None)

x_pos = []
y_pos = []
errors = []

# adding stderr vals to all bars
for i, container in enumerate(ax.containers):
    for bar in container:
        x_pos.append(bar.get_x() + bar.get_width() / 2)
        y_pos.append(bar.get_height())
    errors.append(df.iloc[i]['Stderr'])
ax.errorbar(x_pos, y_pos, yerr=errors, fmt='none', 
            color='black', capsize=5, capthick=1.5, 
            elinewidth=1.5, alpha=0.7, zorder=10)

# i prefer adding bar labels on top of bars to actually see the values
for container in ax.containers:
    if isinstance(container, BarContainer):
        ax.bar_label(container, fmt='%.3f', padding=15, fontsize=9)

ax.set_xlabel('')
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('MATH-500 results (by best checkpoint)', fontsize=14, fontweight='bold')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Set y-axis limit to give room for labels and error bars
ax.set_ylim(0, max(df['Value'] + df['Stderr']) * 1.5)

plt.tight_layout()
plt.show()

### Figure 9: Instruction Following Best checkpoint comparison graphs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.container import BarContainer

df = pd.DataFrame(rows)

# we select only instruction following benchmarks
df = df[df['Benchmark'].isin(['IFEval','IFBench'])]
fig, ax = plt.subplots(figsize=(8, 6))
n_models = len(df['Model'].unique())
palette = sns.color_palette("viridis", n_models)

bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette, errorbar=None)

benchmarks = df['Benchmark'].unique()
models = df['Model'].unique()

for i, container in enumerate(ax.containers):
    if isinstance(container, BarContainer):
        for j, bar in enumerate(container):
            model = models[i]
            benchmark = benchmarks[j]
            stderr = df[(df['Model'] == model) & (df['Benchmark'] == benchmark)]['Stderr'].values[0]
            x = bar.get_x() + bar.get_width() / 2
            y = bar.get_height()
            if y > 0:  # Only add error bars for non-zero values
                ax.errorbar(x, y, yerr=stderr, fmt='none',
                           color='black', capsize=3, capthick=1.5,
                           elinewidth=1.5, alpha=0.7, zorder=10)

# adding value labels on top of bars again
for container in ax.containers:
    if isinstance(container, BarContainer):
        labels = [f'{v.get_height():.3f}' if v.get_height() > 0 else '' for v in container]
        ax.bar_label(container, labels=labels, padding=15, fontsize=8, rotation=0)

ax.set_xlabel('Benchmark (Prompt-level strict accuracy)', fontsize=12)
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Instruction-Following Results\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
# ax.legend(title='Model', frameon=True, fancybox=True, shadow=True, bbox_to_anchor=(1.02, 1), loc='upper left')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Set y-axis limit to give room for labels and error bars
ax.set_ylim(0, max(df['Value'] + df['Stderr']) * 1.25)

plt.tight_layout()
plt.show()

### Figure 10: Safety Best checkpoint comparison graphs

In [ ]:
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

root_file_path = "../safety-eval/generation_results/"
model = "Neelectric/Llama-3.1-8B-Instruct_SFT_Math-220kv00."

safety_benchmark_configs = [
    ('wildguardtest', 'inverted_micro_harm_lower'),
    ('harmbench', 'inverted_micro_asr_lower'),
    ('toxigen', 'safe_overall'),
    ('do_anything_now', 'inverted_macro_asr'),
    ('wildjailbreak:harmful', 'inverted_macro_asr'),
    ('trustllm_jailbreaktrigger', 'inverted_macro_asr')
]
benchmarks = [benchmark[0] for benchmark in safety_benchmark_configs] + ["Average"]

# quick helper function to retrieve values
def retrieve_results(
    results_json_path: str,
    benchmark_configs: list[tuple[str, str]],
    compute_mean: bool = True,
    ):
    """
    Given a string filepath to a JSON file containing results created by safety-eval, and tuples of benchmarks and requested metrics, returns results as list and optionally the mean at the end.

        Args:
            results_json_path (`str`):
                Filepath of safety-eval generated JSON results file.
            benchmark_configs (`list[tuple[str, str]]`):
                List of tuples, each containing a benchmark and associated metric name. 
            compute_mean (`bool`, defaults to `False`):
                Whether or not to append average score at the end.

        Returns:
            Safety benchmark results as a list
    """
    with open(results_json_path) as fp:
        results_dict = json.load(fp)

    scores = []
    for benchmark, score_name in benchmark_configs:
        scores.append(results_dict[benchmark][score_name])
    
    if compute_mean:
        mean = sum(scores) / len(scores)
        scores.append(mean)
        print(f"Average score for {results_json_path} is {mean}")
    return scores


# Base
# Best SFT
# Best Replay
# Best DFT
# Best EMA
# Best CISPO (KL_beta=0.001)
# Best CISPO (KL_beta=0)

### do it for base
base_path = root_file_path + "meta-llama/Llama-3.1-8B-Instruct" + "/base/metrics_base_main.json"
base_values = retrieve_results(results_json_path=base_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
### this is for best low LR sft v00.24
low_lr_path = root_file_path + model + "24" + "/SFT/metrics_SFT_main.json"
low_lr_values = retrieve_results(results_json_path=low_lr_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
### this is best replay v00.15
best_replay_path = root_file_path + model + "15" + "/SFT/metrics_SFT_main.json"
best_replay_values = retrieve_results(results_json_path=best_replay_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
### this needs to become best DFT v00.17
best_dft_path = root_file_path + model + "17" + "/SFT/metrics_SFT_main.json"
best_dft_values = retrieve_results(results_json_path=best_dft_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
### this needs to become best EMA v00.30
best_ema_path = root_file_path + model + "30" + "/SFT/metrics_SFT_main.json"
best_ema_values = retrieve_results(results_json_path=best_ema_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
    
## this needs to become best CISPO with KL
best_cispo_kl_path = root_file_path + "Neelectric/Llama-3.1-8B-Instruct_GRPO_Math-220kv00.12" + "/SFT/metrics_SFT_main-step-000000250.json"
best_cispo_kl_values = retrieve_results(results_json_path=best_cispo_kl_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)

## here we will have best CISPO without KL
best_cispo_path = root_file_path + "Neelectric/Llama-3.1-8B-Instruct_GRPO_Math-220kv00.14" + "/SFT/metrics_SFT_main-step-000000250.json"
best_cispo_values = retrieve_results(results_json_path=best_cispo_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)

# now we put into a dataframe and plot
n = 7
df = pd.DataFrame({
    'Benchmark': benchmarks * n,
    'Value': base_values + low_lr_values + best_replay_values + best_dft_values + best_ema_values + best_cispo_kl_values + best_cispo_values,
    'Model': ['base'] * len(benchmarks) + ['Best low LR SFT (v00.24)'] * len(benchmarks) + ['Best replay (v00.15)'] * len(benchmarks) + ['Best DFT (v00.17)'] * len(benchmarks) + ['Best EMA (v00.25)'] * len(benchmarks) + ['Best CISPO with KL (v00.12)'] * len(benchmarks) + ['Best CISPO (v00.14)'] * len(benchmarks)
})

fig, ax = plt.subplots(figsize=(14, 6))
palette = sns.color_palette("viridis", n)
bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette)

# again we add value labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f', padding=3, fontsize=6)

ax.set_xlabel('')
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Safety Benchmark Results\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# trying to give room for labels
ax.set_ylim(0, 1 * 1.07)

plt.tight_layout()
plt.show()

### Average scores only!

In [ ]:
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

root_file_path = "../safety-eval/generation_results/"
model = "Neelectric/Llama-3.1-8B-Instruct_SFT_Math-220kv00."

safety_benchmark_configs = [
    ('wildguardtest', 'inverted_micro_harm_lower'),
    ('harmbench', 'inverted_micro_asr_lower'),
    ('toxigen', 'safe_overall'),
    ('do_anything_now', 'inverted_macro_asr'),
    ('wildjailbreak:harmful', 'inverted_macro_asr'),
    ('trustllm_jailbreaktrigger', 'inverted_macro_asr')
]
benchmarks = [benchmark[0] for benchmark in safety_benchmark_configs] + ["Average"]

# quick helper function to retrieve values
def retrieve_results(
    results_json_path: str,
    benchmark_configs: list[tuple[str, str]],
    compute_mean: bool = True,
    ):
    """
    Given a string filepath to a JSON file containing results created by safety-eval, and tuples of benchmarks and requested metrics, returns results as list and optionally the mean at the end.

        Args:
            results_json_path (`str`):
                Filepath of safety-eval generated JSON results file.
            benchmark_configs (`list[tuple[str, str]]`):
                List of tuples, each containing a benchmark and associated metric name. 
            compute_mean (`bool`, defaults to `False`):
                Whether or not to append average score at the end.

        Returns:
            Safety benchmark results as a list
    """
    with open(results_json_path) as fp:
        results_dict = json.load(fp)

    scores = []
    for benchmark, score_name in benchmark_configs:
        scores.append(results_dict[benchmark][score_name])
    
    if compute_mean:
        mean = sum(scores) / len(scores)
        scores.append(mean)
        print(f"Average score for {results_json_path} is {mean}")
    return scores


# Base
# Best SFT
# Best Replay
# Best DFT
# Best EMA
# Best CISPO (KL_beta=0.001)
# Best CISPO (KL_beta=0)

### do it for base
base_path = root_file_path + "meta-llama/Llama-3.1-8B-Instruct" + "/base/metrics_base_main.json"
base_values = retrieve_results(results_json_path=base_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
base_values_avg = [base_values[-1]]
print(base_values)

### this is for best low LR sft v00.24
low_lr_path = root_file_path + model + "24" + "/SFT/metrics_SFT_main.json"
low_lr_values = retrieve_results(results_json_path=low_lr_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
low_lr_values_avg = [low_lr_values[-1]]
    
### this is best replay v00.15
best_replay_path = root_file_path + model + "15" + "/SFT/metrics_SFT_main.json"
best_replay_values = retrieve_results(results_json_path=best_replay_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
best_replay_values_avg = [best_replay_values[-1]]

### this needs to become best DFT v00.17
best_dft_path = root_file_path + model + "17" + "/SFT/metrics_SFT_main.json"
best_dft_values = retrieve_results(results_json_path=best_dft_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
best_dft_values_avg = [best_dft_values[-1]]
    
### this needs to become best EMA v00.30
best_ema_path = root_file_path + model + "30" + "/SFT/metrics_SFT_main.json"
best_ema_values = retrieve_results(results_json_path=best_ema_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
best_ema_values_avg = [best_ema_values[-1]]
    
## this needs to become best CISPO with KL
best_cispo_kl_path = root_file_path + "Neelectric/Llama-3.1-8B-Instruct_GRPO_Math-220kv00.12" + "/SFT/metrics_SFT_main-step-000000250.json"
best_cispo_kl_values = retrieve_results(results_json_path=best_cispo_kl_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
best_cispo_kl_values_avg = [best_cispo_kl_values[-1]]

## here we will have best CISPO without KL
best_cispo_path = root_file_path + "Neelectric/Llama-3.1-8B-Instruct_GRPO_Math-220kv00.14" + "/SFT/metrics_SFT_main-step-000000250.json"
best_cispo_values = retrieve_results(results_json_path=best_cispo_path, benchmark_configs=safety_benchmark_configs, compute_mean=True)
best_cispo_values_avg = [best_cispo_values[-1]]

benchmarks = ["Average"]

# now we put into a dataframe and plot
n = 7
df = pd.DataFrame({
    'Benchmark': benchmarks * n,
    'Value': base_values_avg + low_lr_values_avg + best_replay_values_avg + best_dft_values_avg + best_ema_values_avg + best_cispo_kl_values_avg + best_cispo_values_avg,
    'Model': ['base'] * len(benchmarks) + ['Best low LR SFT (v00.24)'] * len(benchmarks) + ['Best replay (v00.15)'] * len(benchmarks) + ['Best DFT (v00.17)'] * len(benchmarks) + ['Best EMA (v00.25)'] * len(benchmarks) + ['Best CISPO with KL (v00.12)'] * len(benchmarks) + ['Best CISPO (v00.14)'] * len(benchmarks)
})

fig, ax = plt.subplots(figsize=(6, 6))
palette = sns.color_palette("viridis", n)
bars = sns.barplot(data=df, x='Benchmark', y='Value', hue='Model', ax=ax, palette=palette)

# again we add value labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f', padding=3, fontsize=6)

ax.set_xlabel('')
ax.set_ylabel('Score (higher is better)', fontsize=12)
ax.set_title('Safety Benchmark Results\n(reporting checkpoint with strongest math performance)', fontsize=14, fontweight='bold')
ax.legend(title='Model', frameon=True, fancybox=True, shadow=True)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# trying to give room for labels
ax.set_ylim(0, 1 * 1.07)

plt.tight_layout()
plt.show()